In [1]:
from models import MotionTransformer

In [2]:
encoder = MotionTransformer(
        input_feats=264,
        num_frames=360,
        num_layers=8,
        latent_dim=512,
        no_clip=False,
        no_eff=False)

/srv/conda/envs/serverai/layout/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [3]:
from utils.get_opt import get_opt

In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
opt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt"
opt = get_opt(opt_path, device)
opt.do_denoise = True

Reading /home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt


In [5]:
import numpy as np
from os.path import join as pjoin
from trainers import DDPMTrainer

mean = np.load(pjoin(opt.meta_dir, 'mean.npy'))
std = np.load(pjoin(opt.meta_dir, 'std.npy'))
trainer = DDPMTrainer(opt, encoder)
trainer.load(pjoin(opt.model_dir, opt.which_epoch + '.tar'))

trainer.eval_mode()
trainer.to(opt.device)

result_dict = {}
with torch.no_grad():
        caption = ["the first thing i like to do on weekends is relaxing and i'll go shopping if i'm not that tired since i started my job i think it's very important to get good sleep during the weekend because when you have work monday to friday the whole week you're very tired so getting a good rest is as important as completing an excellent job "]
        m_lens = torch.LongTensor([360]).to(device)
        pred_motions = trainer.generate(caption, m_lens, 264)
        motion = pred_motions[0].cpu().numpy()
        print(motion.shape)
opt.result_dir = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints"
np.save(pjoin(opt.result_dir, 'motion.npy'), motion)

/srv/conda/envs/serverai/layout/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1000/1000 [00:19<00:00, 50.34it/s]

(360, 264)
